In [2]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets 
import torch.nn.functional as F
import numpy as np
import torch.optim as optim

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
class ConvBlock(nn.Module):
    
    def __init__(self, in_feature, out_feature, num_layers, pool = False):
        super(ConvBlock, self).__init__()
        channels = [in_feature] + [out_feature for i in range(num_layers)]
        layers = []
        for i in range(num_layers):
            layers.append(nn.Conv2d(channels[i], channels[i+1], kernel_size= 3, padding= 1, bias = True))
            layers.append(nn.BatchNorm2d(channels[i+1], affine = True, track_running_stats= True))
            layers.append(nn.ReLU())
            if pool:
                layers.append(nn.MaxPool2d(kernel_size= 2, stride = 2, padding= 0))
            
        self.op = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.op(x)

In [13]:
class Vgg(nn.Module):
    
    def __init__(self):
        super(Vgg, self).__init__()
        self.conv_block1 = ConvBlock(3, 64, 2)
        self.conv_block2 = ConvBlock(64, 128, 2)
        self.conv_block3 = ConvBlock(128, 256, 3)
        self.conv_block4 = ConvBlock(256, 512, 3)
        self.conv_block5 = ConvBlock(512, 512, 3)
        self.fc1 = nn.Linear(in_features= , out_features= 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)
        
    def forward(self, x):
        #feed forward
        x = self.conv_block1(x)
        x = F.max_pool2d(x, kernel_size= 2, stride= 1, padding= 0) # /16
        x = self.conv_block2(x)
        x = F.max_pool2d(x, kernel_size= 2, stride= 1, padding= 0) # /8
        x = self.conv_block3(x)
        x = F.max_pool2d(x, kernel_size= 2, stride= 1, padding= 0) # /4
        x = self.conv_block4(x)
        x = F.max_pool2d(x, kernel_size= 2, stride= 1, padding= 0)
        x = self.conv_block5(x)
        #x = F.max_pool2d(x, kernel_size= 2, stride= 1, padding= 0)
        
        l = l.reshape(-1, 4*4*512)
        l = self.fc1(l)
        l = self.fc2(l)
        out = self.fc3(l)
        
        return F.softmax(out, dim = 1)

In [33]:
class Data():
    
    def __init__(self):
        
        self.train_set = datasets.CIFAR10(
        root = './data',
        train = True,
        download = True,
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        )

        self.test_set = datasets.CIFAR10(
        root= './data',
        train= False,
        download = True,
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        )
        
    def get_data(self):
        return [self.train_set, self.test_set]

In [35]:
data = Data()
train, test = data.get_data()

Files already downloaded and verified
Files already downloaded and verified


In [36]:
train.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']